In [ ]:
import random
import math

total = int(input("얼마를 충전하시겠습니까? "))

jackpot_num_list = []


previous_betting = 0

def make_jackpot_num():
    jackpot_num = 1
    while True:

        num = random.randint(1, 125)
        if num == 1:
                    return jackpot_num
        else:
            jackpot_num += 1


def slot_machine():
    global total, previous_betting, jackpot_num_list

    jackpot_num = make_jackpot_num()

    play_num = 0

    while True:
        betting_str = input("얼마를 배팅 하시겠습니까?")

        if betting_str == '':
            if previous_betting == 0:
                print("이전 배팅 금액이 없습니다. 배팅 금액을 입력해주세요.")
                continue
            else:
                betting = previous_betting
        else:
            try:
                betting = int(betting_str)
                if betting <= 0:
                    print("배팅 금액은 0보다 커야 합니다.")
                    continue
            except ValueError:
                print("유효한 배팅 금액을 입력해주세요.")
                continue

        if betting > total:
            print("배팅 금액이 잔액을 초과합니다.")
            continue

        previous_betting = betting
        play_num += 1
        total -= betting
        probability = (1-math.exp(-(play_num/125)))*100 # 포아송과 기하분포의 cdf로 계산된 확률
        # 20% 기준으로 구간 설정하여 분포확인하기
        intervals = [(1, 28), (29, 64), (65, 115), (116, 200), (200, 1200)]
        last_five = jackpot_num_list[-5:]
        interval_counts = [0] * len(intervals)  # 구간별 카운트를 0으로 초기화
        for num in last_five:
            for i, interval in enumerate(intervals):
                if interval[0] <= num <= interval[1]:  # 숫자가 해당 구간에 속하는지 확인
                    interval_counts[i] += 1
                    break  # 해당 구간에 숫자가 속하면 다음 숫자로 넘어감
        # jackpot_num_list의 길이가 0보다 큰 경우에만 평균을 계산합니다.
        if len(jackpot_num_list) > 0:
            average_5 = round(sum(jackpot_num_list[-5:])/ len(jackpot_num_list[-5:]))
        else:
            average_5 = 0  # 또는 적절한 기본값
        
        print("\n")
        print(f"현재 {play_num}회 배팅 중입니다. total: {total}")
        print(f"구간분포 {interval_counts} 당첨확률{probability:.2f}% 불균도 {(sum(jackpot_num_list)+play_num)/125-len(jackpot_num_list):.2f} 평균{average_5}")
        print(f"(1,28)(29,64)(65,115)(116,200)(201), 이전{jackpot_num_list[-5:]}")
        #print(f"이전 잭팟까지의 횟수 = {jackpot_num_list[-5:]}")

        if play_num == jackpot_num:
            total += betting * 110
            jackpot_num_list.append(play_num)
            average = round(sum(jackpot_num_list) / len(jackpot_num_list))
            print(f"이야~ 대박!🧞‍♂️ 축하합니다! 잭팟입니다! total: {total}, 평균 = {average}")
            print("🧚 ‍ 🧚‍♀️️")
            break

while True:
    slot_machine()

In [ ]:
# 게임 환경 및 보상 시스템 정의
class GameEnvironment:
    def __init__(self):
        # 여기에 게임 환경 초기화 로직
        pass

    def reset(self):
        # 게임 초기 상태를 리턴
        return initial_state

    def step(self, action):
        # 선택된 행동을 게임에 적용
        # 다음 상태와 보상, 게임 종료 여부 반환
        return next_state, reward, done

# 게임 환경 인스턴스 생성
game = GameEnvironment()

# 강화학습 코드에서 사용
def take_action(state, action):
    next_state, reward, done = game.step(action)
    return next_state, reward

def initialize_state():
    return game.reset()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import random

# Q-함수 근사용 신경망 정의
class QNetwork(nn.Module):
    def __init__(self, input_size, output_size):
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, output_size)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)

# 정책 네트워크 정의
class PolicyNetwork(nn.Module):
    def __init__(self, input_size, output_size):
        super(PolicyNetwork, self).__init__()
        self.fc1 = nn.Linear(input_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, output_size)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return torch.softmax(self.fc3(x), dim=-1)

# Q-함수와 정책 네트워크 초기화
q_network = QNetwork(input_size, num_actions)
policy_network = PolicyNetwork(input_size, num_actions)

# 최적화 및 손실 함수 설정
q_optimizer = optim.Adam(q_network.parameters(), lr=learning_rate)
policy_optimizer = optim.Adam(policy_network.parameters(), lr=learning_rate)
mse_loss = nn.MSELoss()

# 학습 루프
for epoch in range(num_epochs):
    state = initialize_state()  # 초기 상태 설정
    episode_reward = 0
    
    for step in range(max_steps):
        # 정책 네트워크를 사용하여 액션 선택
        action_probs = policy_network(state)
        action = torch.multinomial(action_probs, 1).item()
        
        # 선택한 액션을 환경에 적용하고 보상 및 다음 상태 얻기
        next_state, reward = take_action(state, action)
        
        # Q-함수 업데이트
        q_value = q_network(state)[action]
        target_q = reward + gamma * torch.max(q_network(next_state))
        q_loss = mse_loss(q_value, target_q)
        q_optimizer.zero_grad()
        q_loss.backward()
        q_optimizer.step()
        
        # 정책 네트워크 업데이트
        policy_loss = -torch.log(action_probs[action]) * (reward + gamma * torch.max(q_network(next_state)) - q_value)
        policy_optimizer.zero_grad()
        policy_loss.backward()
        policy_optimizer.step()
        
        state = next_state
        episode_reward += reward
    
    print(f"Epoch {epoch+1}, Total Reward: {episode_reward}")

# 학습된 정책을 사용하여 게임 플레이
def play_game(policy_network):
    state = initialize_state()
    episode_reward = 0
    for _ in range(max_steps):
        action_probs = policy_network(state)
        action = torch.argmax(action_probs).item()
        next_state, reward = take_action(state, action)
        episode_reward += reward
        state = next_state
    return episode_reward

# 학습된 정책으로 게임 플레이
final_reward = play_game(policy_network)
print(f"Final Reward: {final_reward}")


# 1. 게임 환경 코드 최적화

In [ ]:
import random
import math

class SlotMachine:
    def __init__(self):
        self.total = 0  # 플레이어의 총 금액
        self.jackpot_num_list = []  # 지난 잭팟 번호들
        self.previous_betting = 0  # 이전 배팅 금액

    def make_jackpot_num(self):
        jackpot_num = 1
        while True:
            num = random.randint(1, 125)
            if num == 1:
                return jackpot_num
            else:
                jackpot_num += 1

    def calculate_probability(self, play_num):
        return (1 - math.exp(-(play_num / 125))) * 100

    def calculate_intervals(self):
        intervals = [(1, 28), (29, 64), (65, 115), (116, 200), (200, 1200)]
        last_five = self.jackpot_num_list[-5:]
        interval_counts = [0] * len(intervals)
        for num in last_five:
            for i, interval in enumerate(intervals):
                if interval[0] <= num <= interval[1]:
                    interval_counts[i] += 1
                    break
        return interval_counts

    def play(self):
        self.total = int(input("얼마를 충전하시겠습니까? "))
        jackpot_num = self.make_jackpot_num()
        play_num = 0

        while True:
            betting = self.input_betting_amount()
            if betting is None:
                continue

            play_num += 1
            self.total -= betting

            probability = self.calculate_probability(play_num)
            interval_counts = self.calculate_intervals()
            average_5 = self.calculate_average_last_5()

            print(f"\n현재 {play_num}회 배팅 중입니다. total: {self.total}")
            print(f"구간분포 {interval_counts} 당첨확률{probability:.2f}%")
            print(f"(1,28)(29,64)(65,115)(116,200)(201), 이전{self.jackpot_num_list[-5:]}")

            if play_num == jackpot_num:
                self.total += betting * 110
                self.jackpot_num_list.append(play_num)
                print(f"이야~ 대박!🧞‍♂️ 축하합니다! 잭팟입니다! total: {self.total}")
                print("🧚 ‍ 🧚‍♀️️")
                break

    def input_betting_amount(self):
        betting_str = input("얼마를 배팅 하시겠습니까? ")
        if betting_str == '':
            if self.previous_betting == 0:
                print("이전 배팅 금액이 없습니다. 배팅 금액을 입력해주세요.")
                return None
            else:
                return self.previous_betting
        else:
            try:
                betting = int(betting_str)
                if betting <= 0 or betting > self.total:
                    print("배팅 금액은 0보다 크고, 잔액을 초과할 수 없습니다.")
                    return None
                self.previous_betting = betting
                return betting
            except ValueError:
                print("유효한 배팅 금액을 입력해주세요.")
                return None


if __name__ == "__main__":
    game = SlotMachine()
    while True:
        game.play()

# DQN모델 생성
- 초기 요령을 알려줘야겠다 그리고 학습한 하이퍼파라미터를 저장하도록 해야겠다.
- 배팅을 얼마단위로 할수있는지도 설정해두어야한다.
- 초기 배팅금 내가 정해 주어야한다.
- 입실론 그리디로 할지 그리디로할지 정해야학 여러가지 학습 방법을 어떻게 정하느냐에따라 성능이 크게 달라진다.

In [ ]:
import random
import math
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# 신경망 구조 정의
class DQN(nn.Module):
    def __init__(self, input_size, hidden_size, action_size):
        super(DQN, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, action_size)
        )

    def forward(self, x):
        return self.network(x)

class SlotMachine:
    # 슬롯 머신 초기화
    def __init__(self):
        self.jackpot_num_list = []

    def make_jackpot_num(self):
        # 잭팟 번호 생성
        jackpot_num = 1
        while True:
            if random.randint(1, 125) == 1:
                return jackpot_num
            else:
                jackpot_num += 1

    def calculate_intervals(self):
        # 구간별 발생 빈도 계산
        intervals = [(1, 28), (29, 64), (65, 115), (116, 200), (200, 1200)]
        last_five = self.jackpot_num_list[-5:]
        interval_counts = [0] * len(intervals)
        for num in last_five:
            for i, interval in enumerate(intervals):
                if interval[0] <= num <= interval[1]:
                    interval_counts[i] += 1
                    break
        return interval_counts

    def calculate_disparity(self, play_num):
        # 불균도 계산
        if len(self.jackpot_num_list) > 0:
            return (sum(self.jackpot_num_list) + play_num) / 125 - len(self.jackpot_num_list)
        else:
            return 0

    def calculate_probability(self, play_num):
        # 당첨 확률 계산
        return (1 - math.exp(-(play_num / 125))) * 100

    def step(self, action, play_num):
        # 슬롯 머신 동작(한 스텝)
        self.jackpot_num_list.append(play_num)  # 잭팟 번호 추가

        # 잭팟 여부 결정 및 보상 계산
        reward = -action  # 기본적으로 배팅 금액만큼 손실
        jackpot_num = self.make_jackpot_num()
        if play_num == jackpot_num:  # 잭팟에 당첨될 경우
            reward += action * 110  # 배팅 금액의 110배 보상

        # 다음 상태 계산
        next_state = [
            self.calculate_intervals(),  # 구간별 발생 빈도
            self.calculate_probability(play_num),  # 당첨 확률
            self.calculate_disparity(play_num)  # 불균도
        ]

        return next_state, reward

# 메인 코드
if __name__ == "__main__":
    # 하이퍼파라미터 설정
    input_size = 7  # 상태 벡터 크기
    hidden_size = 50  # 은닉층 크기
    action_size = 8  # 가능한 행동의 수 (배팅 금액의 수)
    num_episodes = 1000  # 총 에피소드 수
    betting_options = [1, 50, 100, 200, 300, 500, 1000, 2000]  # 가능한 배팅 금액

    # 모델 및 최적화 설정
    policy_net = DQN(input_size, hidden_size, action_size)
    optimizer = optim.Adam(policy_net.parameters())

    # 슬롯 머신 객체 생성
    slot_machine = SlotMachine()

    for episode in range(num_episodes):
        state = [0] * input_size  # 초기 상태
        total_reward = 0  # 총 보상

        for t in range(1000):  # 한 에피소드의 최대 스텝 수
            # 현재 상태로부터 행동 결정(여기서는 무작위 행동)
            action_idx = random.randint(0, action_size-1)  # 0부터 action_size-1까지 무작위 인덱스
            action = betting_options[action_idx]  # 인덱스에 해당하는 배팅 금액 선택

            # 선택된 행동을 슬롯 머신에 적용하고 다음 상태 및 보상 관찰
            next_state, reward = slot_machine.step(action, t+1)

            # 보상 업데이트
            total_reward += reward

            # 신경망 업데이트 로직 (여기서는 생략)
            # ...

            # 다음 상태로 이동
            state = next_state

        print(f"Episode {episode}, Total Reward: {total_reward}")